In [12]:
import torch
import pandas as pd
import plotly.express as px

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("mps")

from sklearn.manifold import TSNE
from dataloader import load_cifar
from models.autoencoder import CIFAR10Autoencoder

model = CIFAR10Autoencoder.load_from_checkpoint("./lightning_logs/cifar10_ae_mse/checkpoints/epoch=199-step=35000.ckpt")
model.eval()

_, _, test_dataloader = load_cifar(batch_size=1)
images, labels = next(iter(test_dataloader))
print("Loaded cifar 10 dataloader")

Files already downloaded and verified
Files already downloaded and verified
Loaded cifar 10 dataloader


In [15]:
encoded_samples = []
model = model.to(device)
for i, (image, label) in enumerate(test_dataloader):
    image = image.to(device)
    label = label.item()

    z = model.get_z(image)
    encoded_img = z.flatten().detach().cpu().numpy()
    encoded_sample = {f"Enc. Variable {i}": enc for i, enc in enumerate(encoded_img)}
    encoded_sample['label'] = label
    encoded_samples.append(encoded_sample)

    # if i == 1000:
    #     break

In [16]:
encoded_samples = pd.DataFrame(encoded_samples)
# print(encoded_samples)
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(encoded_samples.drop(['label'],axis=1))
fig = px.scatter(tsne_results, x=0, y=1,
                    color=encoded_samples.label.astype(str),
                    color_discrete_map={"0":"red", "1":"blue", "2":"yellow", "3":"gray", "4":"brown", "5":"aqua", "6":"maroon", "7":"purple", "8":"teal", "9":"lime"},
                    labels={'0': 'dimension-1', '1': 'dimension-2'})
fig.write_image("img/tsne_new_normal.png")
# fig.show()

/Users/swainsubrat/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/swainsubrat/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

